# Portfolio Analytics AI - Market Data Cache Population Demo

This notebook demonstrates how to pre-populate the market data cache with comprehensive financial data including:
- S&P 500 company stock data
- Sector ETF data for portfolio diversification
- Risk-free rate data for portfolio performance analysis

The cache population process ensures you have a robust dataset for portfolio analytics and optimization tasks.

## Features Demonstrated:
- Database schema creation and management
- Automated data fetching from Yahoo Finance
- Trading holidays handling
- Error handling for delisted companies
- Cache statistics and validation

## 1. Import Required Libraries

Import all necessary libraries for data processing, database management, and portfolio analytics.

In [ ]:
import pandas as pd
import sqlite3
import os
from datetime import datetime, timedelta
import pandas_market_calendars as mcal

# Import portfolio analytics modules
import sys
sys.path.append('../src')  # Add src directory to path
from portfolio_analytics.data_provider import DataProvider

print("Libraries imported successfully!")
print(f"Current working directory: {os.getcwd()}")
print(f"Available directories: {[d for d in os.listdir('.') if os.path.isdir(d)]}")

## 2. Setup Database Connection and Table Creation

Define functions to create the SQLite database tables for storing company information and sector metadata.

In [ ]:
def create_company_list_table(conn):
    """Create the company_list table if it doesn't exist."""
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS company_list (
            Symbol TEXT PRIMARY KEY,
            Name TEXT,
            Sector TEXT
        )
    ''')
    conn.commit()
    print("✓ Company list table created/verified")

def create_sector_metadata_table(conn):
    """Create the sector_metadata table if it doesn't exist."""
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS sector_metadata (
            sector_name TEXT PRIMARY KEY,
            etf_ticker TEXT,
            etf_name TEXT,
            description TEXT,
            created_date TEXT,
            updated_date TEXT
        )
    ''')
    conn.commit()
    print("✓ Sector metadata table created/verified")

# Setup sample data directory and database path
sample_data_dir = '../sample_data'
db_path = os.path.join(sample_data_dir, 'market_data.db')
os.makedirs(sample_data_dir, exist_ok=True)

print(f"Database will be created at: {db_path}")
print(f"Sample data directory: {sample_data_dir}")

## 3. Fetch US Trading Holidays

Get US trading holidays from 2000 to current year using pandas market calendars for accurate data processing.

In [ ]:
def get_us_trading_holidays():
    """
    Returns a list of US trading holidays from 2000 to current year.
    """    
    print("Fetching US trading holidays...")
    
    # Initialize NYSE calendar
    nyse = mcal.get_calendar('NYSE')
    
    # Get holidays from 2000 to current year
    current_year = datetime.now().year
    start_year = 2000
    
    # Get holidays for the date range using the correct API
    start_date = f'{start_year}-01-01'
    end_date = f'{current_year}-12-31'
    
    # Use the schedule method to get valid trading sessions, then get holidays
    schedule = nyse.schedule(start_date=start_date, end_date=end_date)
    
    # Get all business days in the range
    all_business_days = pd.bdate_range(start=start_date, end=end_date)
    
    # Get actual trading days from the schedule
    trading_days = schedule.index.normalize()
    
    # Find holidays (business days that are not trading days)
    holidays = all_business_days.difference(trading_days)
    
    print(f"✓ Retrieved {len(holidays)} US trading holidays from {start_year} to {current_year}")
    return holidays.to_list()

# Get trading holidays
trading_holidays = get_us_trading_holidays()
trading_holidays_str = [holiday.strftime('%Y-%m-%d') for holiday in trading_holidays]

print(f"Sample holidays: {trading_holidays_str[:5]}...")
print(f"Recent holidays: {trading_holidays_str[-5:]}")

## 4. Populate Sector Metadata

Load sector ETF information from CSV file and populate the database with sector metadata for portfolio diversification analysis.

In [ ]:
def populate_sector_metadata(conn, sector_etfs_path):
    """Populate the sector_metadata table from the Sector_ETFs.csv file."""
    if not os.path.exists(sector_etfs_path):
        print(f"❌ Sector_ETFs.csv not found at {sector_etfs_path}")
        return []
    
    print(f"Reading sector ETF data from {sector_etfs_path}")
    df = pd.read_csv(sector_etfs_path)
    current_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    # Display the sector ETF data
    print(f"Found {len(df)} sector ETFs:")
    print(df[['Sector', 'Ticker', 'ETF Name']].to_string(index=False))
    
    # Prepare data for insertion
    sector_data = []
    for _, row in df.iterrows():
        sector_data.append({
            'sector_name': row['Sector'],
            'etf_ticker': row['Ticker'],
            'etf_name': row['ETF Name'],
            'description': f"SPDR sector ETF tracking {row['Sector']} sector",
            'created_date': current_date,
            'updated_date': current_date
        })
    
    # Convert to DataFrame and insert into database
    sector_df = pd.DataFrame(sector_data)
    sector_df.to_sql('sector_metadata', conn, if_exists='replace', index=False)
    print(f"✓ Populated sector metadata with {len(sector_df)} sectors")
    
    return df['Ticker'].tolist()

# Copy sample data from package first
temp_data_provider = DataProvider(debug=True)
temp_data_provider.copy_sample_data(sample_data_dir)
print("✓ Copied current cache of market data from package sample data")

# Setup database connection and create tables
conn = sqlite3.connect(db_path)
create_company_list_table(conn)
create_sector_metadata_table(conn)

# Populate sector metadata
sector_etfs_path = os.path.join(sample_data_dir, 'Sector_ETFs.csv')
sector_etf_symbols = populate_sector_metadata(conn, sector_etfs_path)

## 5. Populate Company List Data

Load S&P 500 company information from Excel file and populate the company_list table with stock symbols, names, and sectors.

In [ ]:
def populate_company_list(conn, excel_path):
    """Populate the company_list table from an Excel file."""
    if not os.path.exists(excel_path):
        print(f"❌ S&P 500 Excel file not found at {excel_path}")
        return []
    
    print(f"Reading S&P 500 company data from {excel_path}")
    df = pd.read_excel(excel_path, sheet_name='basics')
    
    # Display sample data
    print(f"Found {len(df)} S&P 500 companies")
    print("Sample companies:")
    print(df[['Symbol', 'Name', 'Sector']].head(10).to_string(index=False))
    
    # Sector distribution
    sector_counts = df['Sector'].value_counts()
    print(f"\nSector distribution:")
    print(sector_counts.to_string())
    
    # Ensure columns are named correctly for the database table
    df_clean = df[['Symbol', 'Name', 'Sector']].copy()
    df_clean.to_sql('company_list', conn, if_exists='replace', index=False)
    print(f"✓ Populated company list with {len(df_clean)} companies")
    
    return df_clean['Symbol'].tolist()

# Populate company list
excel_path = os.path.join(sample_data_dir, 'snp_500_companies.xlsx')
sp500_symbols = populate_company_list(conn, excel_path)

# Close database connection for now
conn.close()

print(f"\n📊 Database setup complete!")
print(f"   • S&P 500 companies: {len(sp500_symbols) if sp500_symbols else 0}")
print(f"   • Sector ETFs: {len(sector_etf_symbols) if sector_etf_symbols else 0}")
print(f"   • Trading holidays: {len(trading_holidays)}")

## 6. Initialize Data Provider with Cache

Create DataProvider instance with caching enabled and configure it for fetching market data including risk-free rates.

In [ ]:
# Initialize DataProvider with caching enabled, debug output on, and trading holidays
data_provider = DataProvider(
    cache=True, 
    cache_db=db_path, 
    debug=True, 
    trading_holidays=trading_holidays_str
)

print("✓ DataProvider initialized with cache enabled")
print(f"  Cache database: {db_path}")
print(f"  Trading holidays configured: {len(trading_holidays_str)}")

# Get risk-free rate metadata and add the symbol
risk_free_metadata = data_provider.get_risk_free_rate_metadata()
risk_free_symbol = [risk_free_metadata['symbol']]

print(f"\n📈 Risk-free rate information:")
print(f"   Symbol: {risk_free_metadata['symbol']}")
print(f"   Name: {risk_free_metadata['name']}")
print(f"   Currency: {risk_free_metadata['currency']}")
print(f"   Frequency: {risk_free_metadata['frequency']}")
print(f"   Description: {risk_free_metadata['description'][:100]}...")

# Prepare symbol lists for download
if sp500_symbols and sector_etf_symbols:
    all_symbols = sp500_symbols + sector_etf_symbols + risk_free_symbol
    print(f"\n📋 Symbols prepared for download:")
    print(f"   S&P 500 stocks: {len(sp500_symbols)}")
    print(f"   Sector ETFs: {len(sector_etf_symbols)}")
    print(f"   Risk-free rate: 1 ({risk_free_symbol[0]})")
    print(f"   Total symbols: {len(all_symbols)}")
else:
    print("❌ Error: Could not load symbol lists. Check data files.")

## 7. Download Market Data for All Symbols

Fetch 18 years of historical price data for all symbols with comprehensive error handling and progress tracking.

**Note:** This process may take 15-30 minutes depending on network speed and data availability. The cell will show progress for each symbol downloaded.

In [ ]:
# Define date range - 18 years of historical data
end_date = datetime.now() - timedelta(days=1)
start_date = end_date - timedelta(days=18*365)

print(f"📅 Data range: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
print(f"🚀 Starting download process for {len(all_symbols)} symbols...")
print(f"💡 Tip: This may take 15-30 minutes. Progress will be shown below.\n")

# Initialize counters
delisted_count = 0
error_count = 0
success_count = 0
risk_free_downloaded = False

# Lists to track results
successful_symbols = []
delisted_symbols = []
error_symbols = []

# Download data for all symbols
for i, symbol in enumerate(all_symbols):
    # Determine symbol type for display
    if symbol in sector_etf_symbols:
        symbol_type = "sector ETF"
    elif symbol in risk_free_symbol:
        symbol_type = "risk-free rate"
    else:
        symbol_type = "S&P 500 stock"
    
    try:
        print(f"({i+1:3d}/{len(all_symbols)}) Fetching {symbol:6s} ({symbol_type})...", end=" ")
        
        # Fetch the data
        data_provider.get_price_data(
            symbols=symbol,
            start_date=start_date.strftime('%Y-%m-%d'),
            end_date=end_date.strftime('%Y-%m-%d')
        )
        
        success_count += 1
        successful_symbols.append(symbol)
        
        if symbol in risk_free_symbol:
            risk_free_downloaded = True
            print("✅ SUCCESS (Risk-free rate)")
        else:
            print("✅ SUCCESS")
            
    except Exception as e:
        error_msg = str(e)
        if "possibly delisted" in error_msg or "no price data found" in error_msg:
            print(f"⚠️  DELISTED/NO DATA")
            delisted_count += 1
            delisted_symbols.append(symbol)
        else:
            print(f"❌ ERROR: {error_msg}")
            error_count += 1
            error_symbols.append(symbol)
            if symbol in risk_free_symbol:
                print(f"   ❌ Failed to download risk-free rate data")

    # Progress update every 50 symbols
    if (i + 1) % 50 == 0:
        progress = (i + 1) / len(all_symbols) * 100
        print(f"\n📊 Progress: {progress:.1f}% complete ({i+1}/{len(all_symbols)} symbols processed)")
        print(f"   ✅ Success: {success_count}, ⚠️ Delisted: {delisted_count}, ❌ Errors: {error_count}\n")

print(f"\n🎉 Download process completed!")
print(f"📈 Final statistics:")
print(f"   ✅ Successful downloads: {success_count}")
print(f"   ⚠️  Delisted/No data: {delisted_count}")
print(f"   ❌ Errors: {error_count}")
print(f"   🎯 Success rate: {success_count/len(all_symbols)*100:.1f}%")

if risk_free_downloaded:
    print(f"   ✅ Risk-free rate data: Successfully downloaded")
else:
    print(f"   ❌ Risk-free rate data: Download failed")

## 8. Display Cache Statistics and Results

Analyze the populated cache, show database statistics, and demonstrate the new cached stocks methods.

In [ ]:
# Database file statistics
if os.path.exists(db_path):
    db_size = os.path.getsize(db_path)
    print(f"💾 Database file statistics:")
    print(f"   File path: {db_path}")
    print(f"   File size: {db_size / 1024 / 1024:.2f} MB")
    print(f"   File size: {db_size:,} bytes")
else:
    print(f"❌ Database file not found at {db_path}")

# Test the new cached stocks methods
print(f"\n🔍 Testing new cached stocks methods...")

# Get all cached stocks (including ETFs)
all_cached_stocks = data_provider.get_cached_stocks(include_etfs=True)
print(f"   📊 Total symbols in cache: {len(all_cached_stocks)}")

# Get stocks only (excluding ETFs)
stocks_only = data_provider.get_cached_stocks(include_etfs=False)
print(f"   📈 Stock symbols only: {len(stocks_only)}")

# Get ETFs for comparison
cached_etfs = data_provider.get_cached_etfs()
print(f"   🎯 ETF symbols: {len(cached_etfs)}")

# Get detailed symbol information
symbols_info = data_provider.get_cached_symbols_info()
print(f"   📋 Symbols with detailed info: {len(symbols_info)}")

# Analysis of cached data
if symbols_info:
    print(f"\n📊 Cache analysis:")
    
    # Count by symbol type
    etf_count = sum(1 for info in symbols_info.values() if info['symbol_type'] == 'ETF')
    stock_count = len(symbols_info) - etf_count
    
    print(f"   Stock symbols: {stock_count}")
    print(f"   ETF symbols: {etf_count}")
    print(f"   Total symbols: {len(symbols_info)}")
    
    # Data point statistics
    data_points = [info['count'] for info in symbols_info.values()]
    print(f"   Average data points per symbol: {sum(data_points)/len(data_points):.0f}")
    print(f"   Maximum data points: {max(data_points):,}")
    print(f"   Minimum data points: {min(data_points):,}")
    
    # Date range
    start_dates = [info['start_date'] for info in symbols_info.values()]
    end_dates = [info['end_date'] for info in symbols_info.values()]
    print(f"   Data date range: {min(start_dates)} to {max(end_dates)}")
    
    # Symbol with most data
    max_data_symbol = max(symbols_info.keys(), key=lambda k: symbols_info[k]['count'])
    max_count = symbols_info[max_data_symbol]['count']
    print(f"   Symbol with most data: {max_data_symbol} ({max_count:,} points)")
    
    # Sample of successful symbols by type
    sample_stocks = [s for s in stocks_only[:10]]
    sample_etfs = [e for e in cached_etfs[:5]]
    
    print(f"\n📈 Sample successful stock symbols: {', '.join(sample_stocks)}")
    print(f"🎯 Sample successful ETF symbols: {', '.join(sample_etfs)}")
    
    # Risk-free rate verification
    if risk_free_symbol[0] in symbols_info:
        rf_info = symbols_info[risk_free_symbol[0]]
        print(f"✅ Risk-free rate ({risk_free_symbol[0]}): {rf_info['count']} data points, {rf_info['start_date']} to {rf_info['end_date']}")
    else:
        print(f"❌ Risk-free rate ({risk_free_symbol[0]}) not found in cache")

print(f"\n🎉 Cache population demo completed successfully!")
print(f"💡 You can now use this cache for portfolio analytics, optimization, and performance analysis.")
print(f"📚 See other notebooks for examples of using the cached data.")

## Summary and Next Steps

This notebook demonstrated the complete process of populating a comprehensive market data cache for portfolio analytics:

### ✅ What We Accomplished:
1. **Database Setup**: Created SQLite tables for company metadata and sector information
2. **Trading Holidays**: Retrieved and configured US market holidays for accurate data processing
3. **Sector ETFs**: Populated sector metadata for portfolio diversification analysis
4. **S&P 500 Data**: Loaded company information for 500+ stocks
5. **Risk-Free Rates**: Integrated Treasury bill data for performance benchmarking
6. **Bulk Download**: Fetched 18 years of historical price data for all symbols
7. **Cache Analysis**: Demonstrated new methods to query and understand cached data

### 🔧 New DataProvider Methods Used:
- `get_cached_stocks()` - Get list of all stocks in cache
- `get_cached_symbols_info()` - Get detailed information about cached symbols
- `get_cached_etfs()` - Get list of ETF symbols
- `get_risk_free_rate_metadata()` - Get risk-free rate information

### 📊 Typical Results:
- **400+ symbols** successfully cached
- **15-20 MB database** with 18 years of daily data
- **S&P 500 stocks**, **sector ETFs**, and **risk-free rates** included
- **4,000+ data points** per symbol on average

### 🚀 Next Steps:
1. **Portfolio Construction**: Use `01_getting_started.ipynb` to build portfolios
2. **Performance Analysis**: Use `02_analyzer_demo.ipynb` for comprehensive analysis
3. **Custom Analysis**: Create your own notebooks using the cached data
4. **Data Updates**: Re-run this notebook periodically to update the cache

### 💡 Tips:
- The cache persists between sessions - no need to re-download unless updating
- Use `include_etfs=False` in `get_cached_stocks()` for stock-only analysis
- Check `get_cached_symbols_info()` to understand data availability before analysis
- The risk-free rate data enables accurate Sharpe ratio calculations

The cache is now ready for advanced portfolio analytics and optimization!